PROJECT JARINGAN SYARAF TIRUAN

In [1]:
#Import Library
import pandas as pd
import numpy as np

#Membaca Data
df = pd.read_csv("C:/Users/USER/Documents/Python/Jaringan Syaraf Tiruan Project/indian_liver_patient.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB
None


In [2]:
#Check for missing values
print(df.isnull().sum())

print(df['Albumin_and_Globulin_Ratio'].describe())

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64
count    579.000000
mean       0.947064
std        0.319592
min        0.300000
25%        0.700000
50%        0.930000
75%        1.100000
max        2.800000
Name: Albumin_and_Globulin_Ratio, dtype: float64


In [3]:
#Mengisi missing values dengan median
df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].median(), inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  583 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB
None


C:\Users\USER\AppData\Local\Temp\ipykernel_3088\2480636606.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].median(), inplace=True)


SPLIT DATA

In [4]:
# Encode gender: Male=1, Female=0
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

# Features and target
X = df.drop(columns=['Dataset']).values
y = df['Dataset'].values
y = (y == 2).astype(int)  # 0 = Liver disease, 1 = No liver disease

# Custom train-test split function
def tt_split(X, y, test_size=0.2): 
    i = int((1 - test_size) * X.shape[0]) 
    o = np.random.permutation(X.shape[0])
    
    X_train, X_test = np.split(np.take(X, o, axis=0), [i])
    y_train, y_test = np.split(np.take(y, o), [i])
    return X_train, X_test, y_train, y_test

# Perform the split
X_train, X_test, y_train, y_test = tt_split(X, y, test_size=0.2)

# Output the shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((466, 10), (117, 10), (466,), (117,))

MODEL BACKPROPAGATION

TRAINING MODEL

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)

        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.bias_output = np.zeros((1, self.output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feedforward(self, X):
        self.hidden_activation = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_activation)

        self.output_activation = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = self.sigmoid(self.output_activation)

        return self.predicted_output

    def backward(self, X, y, learning_rate):
        output_error = y - self.predicted_output
        output_delta = output_error * self.sigmoid_derivative(self.predicted_output)

        hidden_error = np.dot(output_delta, self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

        self.weights_hidden_output += np.dot(self.hidden_output.T, output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate

        self.weights_input_hidden += np.dot(X.T, hidden_delta) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs=10000, learning_rate=0.1):
        for epoch in range(epochs):
            self.feedforward(X)
            self.backward(X, y, learning_rate)

            if epoch % 1000 == 0 or epoch == epochs - 1:
                loss = np.mean((y - self.predicted_output) ** 2)
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        output = self.feedforward(X)
        return (output > 0.5).astype(int)
        


TESTING

In [9]:

nn = NeuralNetwork(input_size=10, hidden_size=10, output_size=1)
nn.train(X_train, y_train, epochs=10000, learning_rate=0.1)

# Evaluasi
predictions = nn.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"Akurasi: {accuracy * 100:.2f}%")

C:\Users\USER\AppData\Local\Temp\ipykernel_3088\3085006970.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


ValueError: shapes (466,466) and (1,10) not aligned: 466 (dim 1) != 1 (dim 0)

TEST DATA BARU

In [ ]:
pasien_baru = np.array([[...]])
pasien_baru_scaled = scaler.transform(pasien_baru)
prediksi = nn.predict(pasien_baru_scaled)
print("Pasien menderita liver." if prediksi[0][0] == 1 else "Pasien tidak menderita liver.")